In [16]:
#Fit on whole set and predict the labels
import logging
import sys
from tqdm import tqdm
import time 
# import dask.dataframe as dd
#lasso dataset
import pandas as pd
import numpy as np

# 导入库
import shap
import xgboost
import matplotlib.pyplot as plt

def ML_run():
    # logger.info("Start runing...")
    # logger.info("Importing...")
    import warnings
    from sklearn import model_selection
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.ensemble import ExtraTreesClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.metrics import roc_curve, roc_auc_score
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import f1_score
    from sklearn.metrics import auc
    from sklearn.metrics import precision_score, recall_score
    import pickle
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import LeaveOneOut 
    from sklearn.model_selection import cross_val_score

    warnings.filterwarnings('ignore')
    models1 = []
    models1.append(('LogR', LogisticRegression()))
    models1.append(('gNB', GaussianNB()))
    models1.append(('SVM', SVC(kernel = 'rbf', probability=True)))
    models1.append(('DT', DecisionTreeClassifier(criterion = 'entropy', random_state=1)))
    models1.append(('RF', RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)))
    models1.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)))
    models1.append(('LDA', LinearDiscriminantAnalysis()))
    models1.append(('mNB', MultinomialNB()))
    models1.append(('ABC', AdaBoostClassifier()))
    models1.append(('GBC', GradientBoostingClassifier()))
    models1.append(('ETC', ExtraTreesClassifier()))
    models1.append(('BC', BaggingClassifier()))

    #Predict Lables
    # logger.info("Predict Lables")
    predict_lab = []
    predicted_df = pd.DataFrame(columns=[])

    for name, model in models1:

        pickle_files = f'./model/{ab}_{name}.sav'
        #Load the model from disk
        loaded_model = pickle.load(open(pickle_files, 'rb'))
        #Predicting the new data
        pr = pd.read_csv(To_predict)
        Xnew = pr.iloc[:, 1:-1]

        ynew = loaded_model.predict(Xnew)
        predict_lab.append({'predicted label':ynew})
        labels = pd.DataFrame(data=ynew, columns=[f'prediction by {name}'])
        predicted_df = pd.concat([predicted_df, labels], axis=1)
        #Labels predicted my each model    
    Model_Predict = pd.DataFrame(predict_lab)

    #Separate df for each isolate and predicted label by models
    Predicted_Labels = pd.concat([pr['Isolate'], predicted_df], axis=1)  
    file_all_out = f"./Data_to_predict/{ab}_predict_rst.csv"
    
    with open (file_all_out, 'a+') as rf:
        Predicted_Labels.to_csv(rf, index=False)


In [ ]:

import os
all_files = os.listdir()

TXT_files = [file[:-15] for file in all_files if file.endswith('.txt')]

for ab in TXT_files:
    #Import amr-ast data from github repository
    To_predict = f'./Data_to_predict/{ab}_to_predict_MTB_166.csv'
    ML_run()

In [ ]:
import pandas as pd

# Excel files
df = pd.read_excel('rifampicin_predict_rst.xlsx')
def calculate_metrics(y_true, y_pred, label):
    TP = sum((y_true == 1) & (y_pred == 1))
    FP = sum((y_true == 0) & (y_pred == 1))
    TN = sum((y_true == 0) & (y_pred == 0))
    FN = sum((y_true == 1) & (y_pred == 0))

    accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return {f'{label}_TP': TP, f'{label}_FP': FP, f'{label}_TN': TN, f'{label}_FN': FN,
            f'{label}_Accuracy': accuracy, f'{label}_Precision': precision,
            f'{label}_Recall': recall, f'{label}_F1_Score': f1_score}

results = {}
for col in df.columns[2:]:
    y_true = df['True phenotype']
    y_pred = df[col]
    results.update(calculate_metrics(y_true, y_pred, col.replace('prediction by ', '')))

# save results
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Value'])
results_df.to_excel(f'./To_predict_rst_Accuracy/rifampicin_predict_rst_12model.xlsx', index=True)
print(results_df)